## 145: Verificación de Correo Electrónico al Registrarse

Hoy vas a implementar un sistema básico de verificación por correo electrónico, en el que:

✅ Al registrarse, el usuario recibe un correo con un enlace único

🚫 Su cuenta permanece inactiva hasta que verifica su correo

🔐 Esto evita spam, bots o registros inválidos

🧩 1. Ampliar el modelo User con un campo is_active y verification_token
En models.py:

python
Copiar
Editar
from sqlalchemy import Boolean
import secrets

class User(Base):
    __tablename__ = "usuarios"
    id = Column(Integer, primary_key=True, index=True)
    username = Column(String, unique=True)
    email = Column(String, unique=True)
    hashed_password = Column(String)
    rol = Column(String)
    is_active = Column(Boolean, default=False)
    verification_token = Column(String, default=lambda: secrets.token_urlsafe(32))
🧩 2. Actualizar el registro para generar el token y enviar correo
En main.py:

python
Copiar
Editar
from report_utils import enviar_correo_verificacion  # Nueva función
import secrets

@app.post("/register")
def registrar_usuario(datos: RegistroRequest):
    if db.query(User).filter_by(username=datos.username).first():
        raise HTTPException(status_code=400, detail="Nombre de usuario ya registrado.")
    if db.query(User).filter_by(email=datos.email).first():
        raise HTTPException(status_code=400, detail="Correo ya registrado.")

    token = secrets.token_urlsafe(32)
    nuevo_usuario = User(
        username=datos.username,
        email=datos.email,
        hashed_password=User.hashear_password(datos.password),
        rol="usuario",
        verification_token=token,
        is_active=False
    )
    db.add(nuevo_usuario)
    db.commit()

    # Enviar correo
    enviar_correo_verificacion(datos.email, token)

    return {"mensaje": "Registro exitoso. Verifica tu correo para activar la cuenta."}
🧩 3. Ruta de verificación
python
Copiar
Editar
@app.get("/verify/{token}")
def verificar_usuario(token: str):
    user = db.query(User).filter_by(verification_token=token).first()
    if not user:
        raise HTTPException(status_code=404, detail="Token no válido")

    user.is_active = True
    user.verification_token = None
    db.commit()
    return {"mensaje": "✅ Tu cuenta ha sido activada correctamente"}
🧩 4. Función para enviar correo de verificación
En report_utils.py:

python
Copiar
Editar
def enviar_correo_verificacion(destinatario, token):
    remitente = 'tucorreo@gmail.com'
    clave = 'tu_contraseña_app'
    enlace = f"http://localhost:8000/verify/{token}"

    msg = EmailMessage()
    msg['Subject'] = 'Verifica tu cuenta'
    msg['From'] = remitente
    msg['To'] = destinatario
    msg.set_content(f"Hola, por favor verifica tu cuenta dando clic en este enlace: {enlace}")

    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
        smtp.login(remitente, clave)
        smtp.send_message(msg)
🧩 5. Modificar login para validar si is_active es True
python
Copiar
Editar
@app.post("/login")
def login(form_data: OAuth2PasswordRequestForm = Depends()):
    user = db.query(User).filter_by(username=form_data.username).first()
    if not user or not user.verificar_password(form_data.password):
        raise HTTPException(status_code=401, detail="Credenciales inválidas")

    if not user.is_active:
        raise HTTPException(status_code=403, detail="Tu cuenta aún no está verificada")

    token = crear_token({"sub": user.username, "rol": user.rol})
    return {"access_token": token, "token_type": "bearer"}